# Solving Tetris with D-Wave

### In this notebook I attempt to use the D-Wave qpu to generate where to place the next shape in the popular game tetris

### This first cell loads a  python Tetris clone Tetomino with a few modifications and added functions to take input from the D_Wave QPU after the first piece is placed

In [ ]:
from tetris import *

### In the cells below I define my helper functions in order to check the contraints and generate my h and J values

In [ ]:
def check_constraints(results,board,piece,h,static,include_lowest=True):# goes through all samples in results and returns first 
    df = results.to_pandas_dataframe()              #valid sample. if none found returns false
    for key,value in static.items():
        df[key]=[value]*len(df)

    of_interst = []
    for i,row in df[list({**h,**static}.keys())].iterrows():
        if sum(row.values)==num_empty(board)-num_full(board)-8:
                of_interst.append(i)
    df = df[list({**h,**static}.keys())].iloc[of_interst]
    samples_with_4_flip = [dict(row) for i,row in df.iterrows()] 
    flipped = [find_fliped_vars(x,{**h,**static}) for x in samples_with_4_flip]
    samples_with_4_flip = [samp for samp,flip in zip(samples_with_4_flip,flipped) if len(flip)==4]
    flipped = [x for x in flipped if len(x)==4]
    if len(samples_with_4_flip)>0:
        for i,x in enumerate(samples_with_4_flip):
            if include_lowest:
                if is_shape_sample(x,board,piece,flipped[i]) and is_lowest_sample(x):
                    return x   
            else:
                if is_shape_sample(x,board,piece,flipped[i]):
                    return x  
                
    return False

def coord_to_x_rot(coords,piece):# takes the coordinates from flipped and returns the x translation and
    shape = piece['shape']      # rotation in tuple format
    xpos = piece['x']
    rotation = piece['rotation']
    correction ={'S': {0: 1, 1: 2},
                 'Z': {0: 1, 1: 1},
                 'J': {0: 1, 1: 2, 2: 1, 3: 1},
                 'L': {0: 1, 1: 2, 2: 1, 3: 1},
                 'I': {0: 2, 1: 0},
                 'O': {0: 1},
                 'T': {0: 1, 1: 2, 2: 1, 3: 1}}
    valid = shape_to_valid_coord_set(eval(shape+'_SHAPE_TEMPLATE'))
    initial_rot_coords = valid[rotation]
    final_rot = None
    var_to_grid = [tuple(map(int,var.split('_'))) for var in coords]
    var_min_X = min([s[0] for s in var_to_grid])
    var_min_Y = min([s[1] for s in var_to_grid])
    var_patternx = [s[0]-var_min_X for s in var_to_grid] 
    var_patterny = [s[1]-var_min_Y for s in var_to_grid]

    for i,shap in enumerate(valid):
        min_X=min([s[0] for s in shap])
        min_Y=min([s[1] for s in shap])
        patternx = [s[0]-min_X for s in shap]
        patterny = [s[1]-min_Y for s in shap]
        if patternx==var_patternx and patterny==var_patterny:
            print("looped ",i,' times')
            final_rot = i 
            break
        else:
            final_rot=0
    if shape is 'O':
        rotate = 0
        corr = 1
        rot_coords_diff =min([x[0] for x in var_to_grid])-min([x[0] for x in initial_rot_coords])
        x_translate =rot_coords_diff-xpos+corr
    elif shape in ['T','L','J']:
        rotate = len(valid)-1 if final_rot==0 else final_rot-1 
        corr= correction[shape][rotate]  
        rot_coords_diff =min([x[0] for x in var_to_grid])-min([x[0] for x in initial_rot_coords])
        x_translate =rot_coords_diff-xpos
    elif shape in ['S','Z','I']:
        rotate = len(valid)-1 if final_rot==0 else final_rot-1 
        corr= correction[shape][rotate]  
        rot_coords_diff =min([x[0] for x in var_to_grid])-min([x[0] for x in initial_rot_coords])
        x_translate =rot_coords_diff-xpos+corr
    return(x_translate,rotate)

In [ ]:
def generate_h(board,piece,embedding):# generates an h with only non fixed varaibles and no J 
    board_full = [ro [::-1]for ro in board]
    max_height = max([j for i,val in enumerate(board_full) for j,x in enumerate(val) if isinstance(x,int)])
    if max_height <=4:
        max_height=4
    else:
        max_height = max_height
        
    max_width = max([i for i,val in enumerate(board_full) for j,x in enumerate(val) if isinstance(x,int)])
    if max_width <=5:
        max_width=max_width+4
    else:
        max_width =9
    board_ = [ro[:max_height] for ro in board_full][:max_width]
#     shape = PIECES[piece['shape']][piece['rotation']]
#     coords = shape_to_valid_coord_set([shape])
#     coord_combos = valid_coord_to_cobo(coords,board)
    h={}
    for x in range(max_width):
        for y in range(max_height):
             if isinstance(board_[x][y],str):
                    h.update({f'{x}_{y}':-y/max_height})
        
    static = {}
    for x in range(max_width):
        for y in range(max_height):
            if isinstance(board_[x][y],int) and f'{x}_{y}' not in h.keys():
                static.update({f'{x}_{y}':-1})
    for var in embedding.keys():
        if var in h.keys():
            continue
        x,y = map(int,var.split('_'))
        if isinstance(board_full[x][y],int):
            static.update({f'{x}_{y}':-1})
        else:
            static.update({f'{x}_{y}':1})
    return h,{},static

### Now we can launch tetromino and once we place the first piece the QPU will hopefully do the rest

In [ ]:
sampler = DWaveSampler(solver="2000Q_2_1",token='Replace with leap API token')

DW_PARAMS=dict(num_reads=1000,answer_mode='histogram',postprocess='sampling')
main(generate_h,sampler,DW_PARAMS,check_constraints,coord_to_x_rot)# this will launch another window

### If you wanted to test any custom board layouts with the function above you can do so with the cells below

In [ ]:
board = [['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.',
        '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.',
        '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.',
        '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.',
        '.', '.', '.', '.', '.', '.', 1],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.',
        '.', '.', '.', '.', '.', 1, 1],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.',
        '.', '.', '.', '.', '.', '.', 1],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.',
        '.', '.', '.', '.', '.', '.', 1],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.',
        '.', '.', '.', '.', '.', '.', 1],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.',
        '.', '.', '.', '.', '.', '.', 1],
       ['.', '.', '.', '.', '.', '.', '.'
        , '.', '.', '.', '.', '.', '.',
        '.', '.', '.', '.', '.', '.', 1]]
fallingPiece = getNewPiece()
fallingPiece


